各階段設計思路
最後更新：2022/07/10

# **階段三、設計系統(查詢個股)**

查詢股價的部分是使用yahoo網頁來查詢。  
我們這裡先以5871舉例，我們先看看如果系統直接抓span的資料會發生甚麼事情。

In [ ]:
from bs4 import BeautifulSoup 
import requests 
from datetime import datetime
from pytz import timezone   
TPE_time = timezone('Asia/Taipei') #本機時間時區指定

def web(url):#定義網頁查詢函式
    source = requests.get(url, headers={'Connection':'close'}) #連線到指定的網站
    soup = BeautifulSoup(source.content, 'lxml', from_encoding='utf-8') #讀取這個網頁的內容
    return soup #最終結果:回傳網頁內容
stk = 5871 #取資料前4碼
stk = str(stk)
#####查詢最新股價#####
price_url = 'https://tw.stock.yahoo.com/quote/' + stk #查詢股價的網址
span = web(price_url).find_all("span")#抓span資料
print(span)

[<span class="ybar-icon-sprite _yb_1ftij _yb_eisyg"></span>, <span class="_yb_yegfu"></span>, <span class="_yb_zzdzl"></span>, <span>Yahoo奇摩股市應用程式</span>, <span class="ybar-icon-sprite _yb_22p5a _yb_1ftij"></span>, <span class="ybar-icon-sprite _yb_1cuye _yb_1beql"></span>, <span class="_yb_eqayb _yb_13qpd">信箱</span>, <span class="_yb_h7hna" tabindex="0">…</span>, <span class="Mstart(4px)">台股代號查詢</span>, <span class="C(#464e56)">台股美股皆收盤</span>, <span class="C($c-icon) Fz(24px) Mend(20px)">5871</span>, <span class="Mstart(8px)">加入自選股</span>, <span class="Fz(32px) Fw(b) Lh(1) Mend(16px) D(f) Ai(c) C($c-trend-up)">192.5</span>, <span class="Fz(20px) Fw(b) Lh(1.2) Mend(4px) D(f) Ai(c) C($c-trend-up)"><span class="Mend(4px) Bds(s)" style="border-color:transparent transparent #ff333a transparent;border-width:0 6.5px 9px 6.5px"></span>0.5</span>, <span class="Mend(4px) Bds(s)" style="border-color:transparent transparent #ff333a transparent;border-width:0 6.5px 9px 6.5px"></span>, <span class=

我們可以從資料中發現成交價的span，剛好在**`成交span`**後一個。  
因此我們以迴圈來取資料，讓它去抓"成交"的span，並且往後取1個，就會是我們想要的成交價。  
同時我們也發現本益比資料也剛好在同一頁面，因此我們在此順便抓取本益比的資料，只不過抓取的span變成**`成交量span`**，並且往後取1個。

In [ ]:
from bs4 import BeautifulSoup 
import requests 


def web(url):#定義網頁查詢函式
    source = requests.get(url, headers={'Connection':'close'}) #連線到指定的網站
    soup = BeautifulSoup(source.content, 'lxml', from_encoding='utf-8') #讀取這個網頁的內容
    return soup #最終結果:回傳網頁內容
stk = 5871 #取資料前4碼
stk = str(stk)
#####查詢最新股價#####
price_url = 'https://tw.stock.yahoo.com/quote/' + stk #查詢股價的網址
systime = datetime.now(TPE_time) #取得查價時間
span = web(price_url).find_all("span")#抓span資料
for p in span:
    if p.text == '成交':#抓成交sapn
        price = float(span[span.index(p) + 1].string) #股價sapn
        lastprice = price#給值
    elif p.text == '成交量':#抓成交量sapn
        pe_ratio_result = str(span[span.index(p) + 1].string) #本益比sapn
        pe_ratio_result = pe_ratio_result[:5]#取資料前5碼
        pe_ratio_result = float(pe_ratio_result)#轉換格式
    else:
        continue
print(lastprice)
print(pe_ratio_result)

192.5
12.92


接下來根據取得的價格給予相對應的貼圖，讓使用者能夠更快速的明確個股價位

In [ ]:
stk_price = str('')  #股價標籤初始給值
if lastprice >= 100 and lastprice < 250:
    stk_price = str('🟢')
elif lastprice >= 250 and lastprice < 500:
    stk_price = str('🔴')
elif lastprice >= 500:
    stk_price = str('⚫')
print(stk_price)

🟢


接下來重複上述的步驟，但是根據想要取得的資料不同，部分網址及抓取的span等的程式碼會稍微更動。  
特別需要注意的是，有的時候**`季增率span`**或**`股票股利span`**會抓不到，這是取決於當下網站的情況。  
為了避免for迴圈因為抓不到而直接放棄判斷，因此我加上了即使在沒有**`季增率span`**或**`股票股利span`**的情況下，也還是抓地到的`及span`，並對於這部分給予自檢機制。  
若後續程式發現自檢失敗，就不會額外執行計算過程，避免因為錯誤而跳脫迴圈。

In [ ]:
characters = "%," #設定特定字符
#####查詢最新每股盈餘季增率%#####
eps_price_url = 'https://tw.stock.yahoo.com/quote/' + stk + '/eps' #查詢每股盈餘的網址
eps_span = web(eps_price_url).find_all("span") #抓span資料
for p in eps_span:
    if p.text == '季增率%':#抓季增率%sapn
        eps_result = str(eps_span[eps_span.index(p) + 5].string) #季增率%sapn
        for d in range(len(characters)): #個別抓取特定字符
            eps_result = eps_result.replace(characters[d],"") #符合字符替換
        eps_result = float(eps_result) #轉換格式
        eps_yield = eps_result #給值
        self_eps_check1 = 1 #自檢1成功給值
    elif p.text == '及': #抓必定有的span
        eps_non = str(eps_span[eps_span.index(p) + 1].string) #隨便抓
        self_eps_check2 = 1 #自檢2成功給值
    else:
        continue
if self_eps_check2 == 1: #自檢2
    if self_eps_check1 == 1: #自檢1
        eps_check = 1 #eps檢查成功給值
print(eps_yield,self_eps_check1,self_eps_check2)
print(eps_check,"\n=========")
#####查詢最新每股股息(全年)#####
divi_price_url = 'https://tw.stock.yahoo.com/quote/' + stk + '/dividend' #查詢股息的網址
span = web(divi_price_url).find_all("span")#抓span資料
for p in span:
    if p.text == '股票股利':#抓股票股利sapn
        divi_result1 = float(span[span.index(p) + 4].string) #現金股利sapn
        divi_result2 = float(span[span.index(p) + 5].string) #股票股利sapn
        divi_result = divi_result1 + divi_result2 #計算總股息
        stkdiv = divi_result #給值
        self_divi_check1 = 1 #自檢1成功給值
    elif p.text == '及': #抓必定有的span
        divi_non = str(span[span.index(p) + 1].string) #隨便抓
        self_divi_check2 = 1 #自檢2成功給值
    else:
        continue
if self_divi_check2 == 1: #自檢2
    if self_divi_check1 == 1: #自檢1
        divi_check = 1 #divi檢查成功給值
print(stkdiv,self_divi_check1,self_divi_check2)
print(divi_check)

27.65 1 1
1 
6.5 1 1
1


我們用4157為例，看看若抓不到**`股票股利span`**時的結果

In [ ]:
characters = "%," #設定特定字符
stk = str(4157)
self_eps_check1 = 0
self_eps_check2 = 0
eps_check = 0
self_divi_check1 = 0
self_divi_check2 = 0
divi_check = 0
#####查詢最新每股盈餘季增率%#####
eps_price_url = 'https://tw.stock.yahoo.com/quote/' + stk + '/eps' #查詢每股盈餘的網址
eps_span = web(eps_price_url).find_all("span") #抓span資料
for p in eps_span:
    if p.text == '季增率%':#抓季增率%sapn
        eps_result = str(eps_span[eps_span.index(p) + 5].string) #季增率%sapn
        for d in range(len(characters)): #個別抓取特定字符
            eps_result = eps_result.replace(characters[d],"") #符合字符替換
        eps_result = float(eps_result) #轉換格式
        eps_yield = eps_result #給值
        self_eps_check1 = 1 #自檢1成功給值
    elif p.text == '及': #抓必定有的span
        eps_non = str(eps_span[eps_span.index(p) + 1].string) #隨便抓
        self_eps_check2 = 1 #自檢2成功給值
    else:
        continue
if self_eps_check2 == 1: #自檢2
    if self_eps_check1 == 1: #自檢1
        eps_check = 1 #eps檢查成功給值
print(eps_yield,self_eps_check1,self_eps_check2)
print(eps_check,"\n=========")
#####查詢最新每股股息(全年)#####
divi_price_url = 'https://tw.stock.yahoo.com/quote/' + stk + '/dividend' #查詢股息的網址
span = web(divi_price_url).find_all("span")#抓span資料
for p in span:
    if p.text == '股票股利':#抓股票股利sapn
        divi_result1 = float(span[span.index(p) + 4].string) #現金股利sapn
        divi_result2 = float(span[span.index(p) + 5].string) #股票股利sapn
        divi_result = divi_result1 + divi_result2 #計算總股息
        stkdiv = divi_result #給值
        self_divi_check1 = 1 #自檢1成功給值
    elif p.text == '及': #抓必定有的span
        divi_non = str(span[span.index(p) + 1].string) #隨便抓
        self_divi_check2 = 1 #自檢2成功給值
    else:
        continue
if self_divi_check2 == 1: #自檢2
    if self_divi_check1 == 1: #自檢1
        divi_check = 1 #divi檢查成功給值
print(stkdiv,self_divi_check1,self_divi_check2)
print(divi_check)

73.68 1 1
1 
6.5 0 1
0


再來我們要向證交所取得個股資料，從中獲取過去30日的成交價及交易量。  
這裡我們使用**`twstock套件`**來抓取。  
同時透過**`twstock`**來簡單做基礎技術分析。

In [ ]:
import twstock as t
stk = 5871 #取資料前4碼
stk = str(stk)
divi_check = 1
eps_check = 1
#####取得個股各項資料#####
stock = t.Stock(stk) #送出查詢至twse並取得回傳資料
price = stock.price#從回傳資料取價格並給值
mixresult = str('查無計算所需資料') #mixresult初始給值      
if divi_check == 1: #divi檢查成功
    if eps_check == 1: #eps檢查成功
        b = t.BestFourPoint(stock)#從回傳資料分析四大買賣點
        mixresult = b.best_four_point() #綜合分析 
        result = mixresult[0]#從分析取得買賣點
print("price=====\n",price)
print("mixresult=====\n",mixresult)
print("result=====\n",result)

price=====
 [199.0, 210.0, 215.5, 225.0, 222.5, 222.5, 222.5, 223.0, 231.0, 228.5, 223.5, 221.0, 217.5, 226.5, 227.0, 218.5, 216.0, 224.5, 202.5, 203.0, 205.0, 212.5, 214.5, 217.0, 208.5, 189.0, 188.0, 191.0, 190.5, 192.0, 192.5]
mixresult=====
 (True, '三日均價大於六日均價')
result=====
 True


再來是計算28MA，供後續判斷。  
我們可以從上面的輸出結果看到，price的取值為近30日的結果，因此我們從資料中直接算28均線的情況下，我們將會有4個值。

In [ ]:
#####計算28MA#####
average = stock.moving_average(price, 28)#取價並依序計算並給值
print(average)

[214.46, 214.16, 213.52, 212.7]


之後，我們再取最後一個值，做為28MA判斷基準。  
並重新編排時間格式，以利後續輸出。 

In [ ]:

last28price = average[3]#從值取第4位價格
time = systime.strftime('%Y/%m/%d,%H:%M:%S')#重新編時間格式
print(last28price)

212.7


為了更進階的篩選股票，因此我也將成交量納入判斷標準，並且比書中作者的判斷更嚴格。   
我的判斷是**`近一日的成交量要大於過去5日的最大量`**，因此這邊我們需要取的過去的成交量，並加以取出其中的最大量。

In [ ]:
#####查詢成交量#####
amount = stock.capacity#從回傳資料取成交量並給值
lastamu = amount[30]#從值取第31位成交量
recentamu = amount[26:]#從值取後5日成交量
maxamu = max(recentamu)#從值取後5日最大成交量
print("amount=====\n",amount)
print("recentamu=====\n",recentamu)
print("maxamu=====\n",maxamu)

amount=====
 [7333919, 4843120, 4502326, 10620143, 3706543, 2300547, 2293088, 2379271, 3257592, 3154608, 3595028, 3578684, 3825636, 4439923, 6829218, 5126725, 3724926, 3812296, 5022579, 3743324, 3999197, 4152435, 3379682, 4499644, 5150110, 16773347, 7397566, 5092864, 6975961, 6107200, 6622902]
recentamu=====
 [7397566, 5092864, 6975961, 6107200, 6622902]
maxamu=====
 7397566
